In [1]:
import pandas as pd
import numpy as np

In [2]:
x_train=pd.read_csv('/Users/zhilinyang/Desktop/work/x_train1.csv')
x_train=x_train.iloc[:,1:]
x_train.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,Z304HVAM,74048288.0,95.0,100.0,84.0,89.0,100.0,100.0,26.0,100.0,0.0,0.0
1,S300XRA6,100065256.0,96.0,100.0,89.0,78.0,100.0,100.0,19.0,100.0,0.0,0.0
2,Z304JMSQ,75736472.0,92.0,100.0,87.0,85.0,100.0,100.0,27.0,100.0,0.0,0.0
3,Z302B0HC,242383000.0,92.0,100.0,89.0,78.0,100.0,100.0,30.0,100.0,0.0,0.0
4,Z300MG4B,44841392.0,92.0,100.0,75.0,64.0,100.0,88.0,22.0,100.0,0.0,0.0


In [3]:
x_test=pd.read_csv('/Users/zhilinyang/Desktop/work/x_test1.csv')
x_test=x_test.iloc[:,1:]
x_test.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,W300R8FD,210428384.0,96.0,100.0,72.0,75.0,100.0,100.0,21.0,100.0,0.0,0.0
1,Z304JMTY,185917944.0,97.0,100.0,88.0,82.0,100.0,100.0,22.0,100.0,0.0,0.0
2,Z301JZLE,235594800.0,93.0,100.0,84.0,90.0,100.0,100.0,22.0,100.0,0.0,0.0
3,S300XE2H,79078632.0,100.0,100.0,90.0,76.0,100.0,100.0,20.0,100.0,0.0,0.0
4,S300Z5LH,167795152.0,97.0,100.0,79.0,76.0,100.0,100.0,27.0,100.0,0.0,0.0


In [4]:
y_train=pd.read_csv('/Users/zhilinyang/Desktop/work/y_train1.csv',header=None)
y_train=y_train.iloc[:,1:]
y_train.head()

,1
0,0
1,0
2,0
3,0
4,0


In [5]:
y_test=pd.read_csv('/Users/zhilinyang/Desktop/work/y_test1.csv',header=None)
y_test=y_test.iloc[:,1:]
y_test.head()

,1
0,0
1,0
2,1
3,0
4,0


In [35]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier(criterion='entropy',min_samples_split = 20, min_samples_leaf = 7, min_impurity_decrease = 0.001)

In [36]:
len(y_train)

111159

In [37]:
DT.fit(x_train.iloc[:,1:],y_train.values)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [38]:
DT.score(x_test.iloc[:,1:],y_test.values)

0.91885698118545767

In [39]:
y_pred=DT.predict(x_test.iloc[:,1:])

In [40]:
from sklearn.metrics import confusion_matrix
def FDR(y_true, y_pred):
    mat=confusion_matrix(y_true=y_true,y_pred=y_pred)
    TP=mat[1,1]
    FN=mat[1,0]
    FDR=TP/(TP+FN)
    return FDR

In [41]:
FDR(y_test,y_pred)

0.510416666667


0.51041666666666663

In [42]:
def FAR(y_true, y_pred):
    mat=confusion_matrix(y_true=y_true,y_pred=y_pred)
    FP=mat[0,1]
    TN=mat[0,0]
    FAR=FP/(FP+TN)
    return FAR

In [43]:
FAR(y_test,y_pred)

0.066590485120181273

In [44]:
def predict_disk(testset,y_pred,disk_name_list):
    result=[]
    test_set=testset.groupby('serial_number')
    y_pred=pd.Series(y_pred)
    for disk in disk_name_list:
        sample_index=test_set.get_group(disk).index
        sample_result=y_pred.reindex(sample_index)
        sample_result=pd.Series(sample_result)
        if sample_result.any():
            result.append(1)
        else:
            result.append(0)
    return result

In [45]:
def disk_state(testset,y_true,disk_name_list):
    result=[]
    test_set=testset.groupby('serial_number')
    for disk in disk_name_list:
        sample_index=test_set.get_group(disk).index
        sample_result=y_true.reindex(sample_index)
        sample_result=sample_result.values
        if sample_result.sum():
            result.append(1)
        else:
            result.append(0)
    return result

In [46]:
def disk_FDR(x_test,y_pred,y_test):
    disk_pred=predict_disk(x_test,y_pred,x_test['serial_number'].unique())
    true_state=disk_state(x_test,y_test,x_test['serial_number'].unique())
    true_state=np.array(true_state)
    disk_pred=np.array(disk_pred)
    diskfdr=FDR(true_state, disk_pred)
    return diskfdr

In [47]:
disk_FDR(x_test,y_pred,y_test)

0.634868421053


0.63486842105263153

In [48]:
def disk_FAR(x_test,y_pred,y_test):
    disk_pred=predict_disk(x_test,y_pred,x_test['serial_number'].unique())
    true_state=disk_state(x_test,y_test,x_test['serial_number'].unique())
    true_state=np.array(true_state)
    disk_pred=np.array(disk_pred)
    diskfdr=FAR(true_state, disk_pred)
    return diskfdr

In [49]:
disk_FAR(x_test,y_pred,y_test)

0.082041665445808556

In [16]:
new_list=pd.read_csv('/Users/zhilinyang/Desktop/work/new_list.csv')
new_list=new_list.iloc[:,1]
new_list.head()

0    S300XNYV
1    W300SPHE
2    Z305D6HR
3    Z3029FM3
4    W300BH1H
Name: S300XPAS, dtype: object

In [21]:
len(new_list)

3412

In [19]:
fail_list=pd.read_csv('/Users/zhilinyang/Desktop/work/failed_samples.csv')
fail_list=fail_list['serial_number'].unique()
fail_list

array(['Z300KMM3', 'Z302DK2R', 'Z304H3GQ', ..., 'Z304919P', 'Z30671D8',
       'Z3047S8J'], dtype=object)

In [20]:
len(fail_list)

1058

In [23]:
new_list=new_list.append(pd.Series(fail_list))
len(new_list)

4470

In [30]:
def downsampling(dataset,y,name_list):
    dataset=pd.concat([dataset,y],axis=1)
    dataset=dataset[dataset['serial_number'].isin(name_list)]
    print(len(dataset))
    return dataset   

In [39]:
train=downsampling(x_train,y_train,new_list)
test=downsampling(x_test,y_test,new_list)

19008
13884


In [50]:
new_x_train=train.iloc[:,:-1]
new_x_test=test.iloc[:,:-1]
new_x_test.head()

NameError: name 'train' is not defined

In [41]:
new_y_train=train.iloc[:,-1]
new_y_test=test.iloc[:,-1]
new_y_test

2         1
26        0
28        0
39        1
48        0
68        0
69        1
85        0
86        0
88        1
89        0
93        0
116       0
120       1
144       0
148       0
173       1
181       0
203       0
204       0
208       0
212       1
216       0
221       0
236       0
240       0
241       0
249       0
251       1
253       0
         ..
105822    0
105835    0
105838    1
105843    0
105862    0
105870    0
105871    0
105879    1
105887    1
105890    0
105914    0
105918    0
105922    0
105933    1
105946    0
105947    0
105956    0
105961    0
105965    0
105969    0
105977    0
105980    0
105991    1
106008    1
106015    0
106016    1
106021    1
106023    0
106030    0
106033    1
Name: 1, Length: 13884, dtype: int64

In [15]:
DT.fit(new_x_train.iloc[:,1:],new_y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [16]:
new_y_pred=DT.predict(new_x_test.iloc[:,1:])

In [18]:
from sklearn.metrics import precision_score
print('precision: %.3f' % precision_score(y_true=new_y_test.values,y_pred=new_y_pred))

precision: 0.466


In [48]:
new_x_train.to_csv('/Users/zhilinyang/Desktop/work/new_x_train.csv')
print('done')

done


In [49]:
new_y_train.to_csv('/Users/zhilinyang/Desktop/work/new_y_train.csv')
print('done')

done


In [50]:
new_x_test.to_csv('/Users/zhilinyang/Desktop/work/new_x_test.csv')
print('done')

done


In [51]:
new_y_test.to_csv('/Users/zhilinyang/Desktop/work/new_y_test.csv')
print('done')

done


In [51]:
new_x_train=pd.read_csv('/Users/zhilinyang/Desktop/work/new_x_train.csv')
new_x_train=new_x_train.iloc[:,1:]
new_x_train.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,Z304HVAM,74048288.0,95.0,100.0,84.0,89.0,100.0,100.0,26.0,100.0,0.0,0.0
1,Z305DJ1J,126636544.0,99.0,100.0,78.0,96.0,100.0,100.0,24.0,100.0,0.0,0.0
2,S300Z567,95737576.0,93.0,100.0,89.0,80.0,100.0,100.0,36.0,100.0,0.0,0.0
3,Z302B0H5,159956032.0,92.0,100.0,89.0,76.0,100.0,100.0,21.0,100.0,0.0,0.0
4,Z305D4MM,140357160.0,92.0,100.0,87.0,84.0,100.0,100.0,35.0,100.0,0.0,0.0


In [52]:
new_y_train=pd.read_csv('/Users/zhilinyang/Desktop/work/new_y_train.csv',header=None)
new_y_train=new_y_train.iloc[:,1]
new_y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: 1, dtype: int64

In [53]:
new_x_test=pd.read_csv('/Users/zhilinyang/Desktop/work/new_x_test.csv')
new_x_test=new_x_test.iloc[:,1:]
new_x_test.head()

,serial_number,smart_1_raw,smart_3_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_187_normalized,smart_189_normalized,smart_194_normalized,smart_197_normalized,smart_5_raw,smart_197_raw
0,Z301JZLE,235594800.0,93.0,100.0,84.0,90.0,100.0,100.0,22.0,100.0,0.0,0.0
1,Z302ENK8,129171640.0,91.0,100.0,87.0,82.0,100.0,100.0,25.0,100.0,0.0,0.0
2,Z300X9B0,58641392.0,91.0,100.0,81.0,61.0,100.0,89.0,25.0,100.0,0.0,0.0
3,Z300GZD3,4671680.0,91.0,100.0,82.0,59.0,100.0,98.0,25.0,100.0,0.0,0.0
4,W300A1HY,91756984.0,91.0,100.0,84.0,56.0,100.0,95.0,26.0,100.0,0.0,0.0


In [54]:
new_y_test=pd.read_csv('/Users/zhilinyang/Desktop/work/new_y_test.csv',header=None)
new_y_test=new_y_test.iloc[:,1:]
new_y_test.head()

,1
0,1
1,0
2,0
3,1
4,0


In [60]:
from sklearn.tree import DecisionTreeClassifier
DT=DecisionTreeClassifier(criterion='entropy',min_samples_split = 20, min_samples_leaf = 7, min_impurity_decrease = 0.001)

In [61]:
DT.fit(new_x_train.iloc[:,1:],new_y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.001, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=20,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [62]:
new_y_pred=DT.predict(new_x_test.iloc[:,1:])

In [63]:
FDR(new_y_test.values,new_y_pred)

0.654605263158


0.65460526315789469

In [64]:
FAR(new_y_test.values,new_y_pred)

0.26729191090269638

In [66]:
disk_FDR(new_x_test,new_y_pred,new_y_test)

0.773026315789


0.77302631578947367

In [67]:
disk_FAR(new_x_test,new_y_pred,new_y_test)

0.38774912075029311

In [28]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop

In [29]:
model = Sequential([
    Dense(11, input_dim=11),
    Activation('relu'),
    Dense(100),
    Activation('softmax'),
    Dense(1),
    Activation('softmax'),
])

In [30]:
rmsprop = RMSprop(lr=0.1, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(new_x_train.iloc[:,1:], new_y_train, nb_epoch=4, batch_size=1000)

/Users/zhilinyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/4
19008/19008 [==============================] - 0s 16us/step - loss: 8.5851 - acc: 0.4615
Epoch 2/4
19008/19008 [==============================] - 0s 3us/step - loss: 8.5851 - acc: 0.4615
Epoch 3/4
19008/19008 [==============================] - 0s 3us/step - loss: 8.5851 - acc: 0.4615
Epoch 4/4
19008/19008 [==============================] - 0s 4us/step - loss: 8.5851 - acc: 0.4615


In [32]:
loss, accuracy = model.evaluate(new_x_test.iloc[:,1:], new_y_test)

print('test loss: ', loss)
print('test accuracy: ', accuracy)

13884/13884 [==============================] - 0s 18us/step
test loss:  11.7535477451
test accuracy:  0.262748487459


In [68]:
y_pred_nn=model.predict(new_x_test.iloc[:,1:])

In [72]:
disk_FDR(new_x_test,y_pred_nn.flatten(),new_y_test)

1.0


1.0

In [73]:
disk_FAR(new_x_test,y_pred_nn.flatten(),new_y_test)

1.0